In [46]:
%pip install recbole ray >> None
%pip install kmeans_pytorch >>None


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle

import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity


In [5]:
import warnings
warnings.filterwarnings('ignore')

## Get data

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

In [7]:
interactions_df = pd.read_csv('../data_original/interactions.csv')
users_df = pd.read_csv('../data_original/users.csv')
items_df = pd.read_csv('../data_original/items.csv')

In [8]:
interactions_df['t_dat'] = pd.to_datetime(interactions_df['last_watch_dt'], format="%Y-%m-%d")
interactions_df['timestamp'] = interactions_df.t_dat.values.astype(np.int64) // 10 ** 9

In [9]:
df = interactions_df[['user_id', 'item_id', 'timestamp']].rename(
    columns={'user_id': 'user_id:token', 'item_id': 'item_id:token', 'timestamp': 'timestamp:float'})

In [10]:
%mkdir recbox_data

In [11]:
df.to_csv('recbox_data/recbox_data.inter', index=False, sep='\t')

In [ ]:
%pip install ray[tune] >>None
%pip install pyarrow >>None

In [16]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec, Caser
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
from recbole.quick_start import run_recbole

2024-03-13 14:41:00,081	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [17]:
parameter_dict = {
    'data_path': '',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'device': 'GPU',
    'user_inter_num_interval': "[40,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'neg_sampling': None,
    'epochs': 10,
    'eval_args': {
        'split': {'RS': [9, 0, 1]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}
}
config = Config(model='MultiVAE', dataset='recbox_data', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
# logger.info(config)

In [18]:
dataset = create_dataset(config)
logger.info(dataset)

13 Mar 14:41    INFO  recbox_data
The number of users: 13355
Average actions of users: 63.815710648494836
The number of items: 3294
Average actions of items: 258.78985727300335
The number of inters: 852195
The sparsity of the dataset: 98.06281322904924%
Remain Fields: ['user_id', 'item_id', 'timestamp']
recbox_data
The number of users: 13355
Average actions of users: 63.815710648494836
The number of items: 3294
Average actions of items: 258.78985727300335
The number of inters: 852195
The sparsity of the dataset: 98.06281322904924%
Remain Fields: ['user_id', 'item_id', 'timestamp']


In [19]:
# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

13 Mar 14:41    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
13 Mar 14:41    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}]


In [20]:
import time

# Протестируем 3 модели: BPR, CDAE, MultiVAE

In [21]:
%%time
model_list = ['BPR', 'RecVAE', 'MacridVAE', 'CDAE']
results = {}

for model_name in model_list:
    print(f"running {model_name}...")
    start = time.time()
    results[model_name] = run_recbole(model=model_name, dataset = 'recbox_data', config_dict = parameter_dict)
    t = time.time() - start
    print(f"It took {t/60:.2f} mins")
    print(results[model_name])

running BPR...


13 Mar 14:41    INFO  ['/home/beartk/Documents/Recomendations-project/.venv/lib/python3.10/site-packages/ipykernel_launcher.py', '--f=/home/beartk/.local/share/jupyter/runtime/kernel-v2-97305Tc7eAAMavsGd.json']
['/home/beartk/Documents/Recomendations-project/.venv/lib/python3.10/site-packages/ipykernel_launcher.py', '--f=/home/beartk/.local/share/jupyter/runtime/kernel-v2-97305Tc7eAAMavsGd.json']
13 Mar 14:41    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1


stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = item_id
RATING_FIELD = rating
TIME_FIELD = timestamp
seq_len = None
LABEL_FIELD = label
threshold = None
NEG_PREFIX = neg_
load_col = {'inter': ['user_id', 'item_id', 'timestamp']}
unload_col = None
unused_col = None
additional_feat_suffix = None
rm_dup_inter = None
val_interval = None
filter_inter_by_user_or_item = True
user_inter_num_interval = [40,inf)
item_inter_num_interval = [40,inf)
alias_of_user_id = None
alias_of_item_id = None
alias_of_entity_id = None
alias_of_re

It took 1.69 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0824), ('mrr@10', 0.1716), ('ndcg@10', 0.0819), ('hit@10', 0.3479), ('precision@10', 0.0457)])}
running RecVAE...


13 Mar 14:43    INFO  ['/home/beartk/Documents/Recomendations-project/.venv/lib/python3.10/site-packages/ipykernel_launcher.py', '--f=/home/beartk/.local/share/jupyter/runtime/kernel-v2-97305Tc7eAAMavsGd.json']
['/home/beartk/Documents/Recomendations-project/.venv/lib/python3.10/site-packages/ipykernel_launcher.py', '--f=/home/beartk/.local/share/jupyter/runtime/kernel-v2-97305Tc7eAAMavsGd.json']
13 Mar 14:43    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay =

It took 3.23 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0849), ('mrr@10', 0.1697), ('ndcg@10', 0.0828), ('hit@10', 0.3532), ('precision@10', 0.047)])}
running MacridVAE...


13 Mar 14:46    INFO  ['/home/beartk/Documents/Recomendations-project/.venv/lib/python3.10/site-packages/ipykernel_launcher.py', '--f=/home/beartk/.local/share/jupyter/runtime/kernel-v2-97305Tc7eAAMavsGd.json']
['/home/beartk/Documents/Recomendations-project/.venv/lib/python3.10/site-packages/ipykernel_launcher.py', '--f=/home/beartk/.local/share/jupyter/runtime/kernel-v2-97305Tc7eAAMavsGd.json']
13 Mar 14:46    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay =

It took 4.12 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0836), ('mrr@10', 0.1563), ('ndcg@10', 0.0784), ('hit@10', 0.3504), ('precision@10', 0.0458)])}
running CDAE...


13 Mar 14:50    INFO  ['/home/beartk/Documents/Recomendations-project/.venv/lib/python3.10/site-packages/ipykernel_launcher.py', '--f=/home/beartk/.local/share/jupyter/runtime/kernel-v2-97305Tc7eAAMavsGd.json']
['/home/beartk/Documents/Recomendations-project/.venv/lib/python3.10/site-packages/ipykernel_launcher.py', '--f=/home/beartk/.local/share/jupyter/runtime/kernel-v2-97305Tc7eAAMavsGd.json']
13 Mar 14:50    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay =

It took 0.96 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0634), ('mrr@10', 0.1439), ('ndcg@10', 0.0659), ('hit@10', 0.2723), ('precision@10', 0.0344)])}
CPU times: user 46min 32s, sys: 1min 5s, total: 47min 38s
Wall time: 10min


In [38]:
result = {'model': list(results.keys())}
for key in results['BPR']['test_result'].keys():
    result[key] = [results[model]['test_result'][key] for model in result['model']]

In [39]:
result_df = pd.DataFrame(result)

In [40]:
result_df

,model,recall@10,mrr@10,ndcg@10,hit@10,precision@10
0,BPR,0.0824,0.1716,0.0819,0.3479,0.0457
1,RecVAE,0.0849,0.1697,0.0828,0.3532,0.0470
2,MacridVAE,0.0836,0.1563,0.0784,0.3504,0.0458
3,CDAE,0.0634,0.1439,0.0659,0.2723,0.0344


In [45]:
pivot_results = (result_df.groupby(["model"], sort=False).agg(["mean"]))
mean_metric_subset = [(metric, "mean") for metric in pivot_results.columns.levels[0]]
(
    pivot_results.style
    .highlight_min(subset=mean_metric_subset, color='coral', axis=0)
    .highlight_max(subset=mean_metric_subset, color='green', axis=0)
)

,recall@10,mrr@10,ndcg@10,hit@10,precision@10
,mean,mean,mean,mean,mean
model,,,,,
BPR,0.082400,0.171600,0.081900,0.347900,0.045700
RecVAE,0.084900,0.169700,0.082800,0.353200,0.047000
MacridVAE,0.083600,0.156300,0.078400,0.350400,0.045800
CDAE,0.063400,0.143900,0.065900,0.272300,0.034400


По метрикам модель RecVAE наиболее качественная, BPR и MultiVAE отстают немного, CDAE использовать не стоит в данном случае.